In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import torch

data = torch.load("gdrive/MyDrive/preprocessed_images_with_metadata_and_target.pt.nosync")

In [ ]:
def flip_image(image):
  left_side_count = np.count_nonzero(image[:, :image.shape[1] // 2])
  right_side_count = np.count_nonzero(image[:, image.shape[1] // 2:])

  if left_side_count < right_side_count:
      flipped_image = np.fliplr(image)
  else:
      flipped_image = image
  return flipped_image

In [ ]:
features = []
metadata = []
y = []

perm = torch.randperm(len(data))
idx = perm[:700]

for id in idx:
    [target, age, implant], image = data[id]
    features.append(flip_image(image.numpy()))
    metadata.append((age, implant))
    y.append(target)
del data, idx, perm

In [ ]:
f_np = np.array(features).reshape((-1, 1, 512, 512))
features = torch.tensor(f_np).float()
del f_np
m_np = np.asarray(metadata).reshape((-1, 2))
metadata = torch.tensor(m_np).float()
del m_np
y = torch.tensor(y).view(-1, 1).float()

import gc
gc.collect()

9

In [ ]:
features.shape

torch.Size([700, 1, 512, 512])

In [ ]:
metadata.shape

torch.Size([700, 2])

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(features, metadata, y, test_size=0.15,
                                                                      random_state=3451)

train_dataset = TensorDataset(X1_train, X2_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=16)

val_dataset = TensorDataset(X1_val, X2_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=16)

In [ ]:
def probabilistic_f1(labels, predictions, beta=1):
    pTP = np.sum(labels * predictions)
    pFP = np.sum((1 - labels) * predictions)
    num_positives = np.sum(labels)  #  = pTP+pFN

    pPrecision = pTP / (pTP + pFP)
    pRecall = pTP / num_positives

    beta_squared = beta ** 2

    if pPrecision > 0 and pRecall > 0:
        pF1 = (1 + beta_squared) * pPrecision * pRecall / (beta_squared * pPrecision + pRecall)
        return pF1
    else:
        return 0

In [ ]:
import torch
from torch import nn

class Encoder(nn.Module):
    def __init__(self, in_channels):
        super(Encoder, self).__init__()
        self.enc1 = self.conv_block(in_channels, 32)
        self.enc2 = self.conv_block(32, 64)
        self.enc3 = self.conv_block(64, 128)
        self.enc4 = self.conv_block(128, 1024)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=9, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=6, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(enc1)
        enc3 = self.enc3(enc2)
        enc4 = self.enc4(enc3)
        return enc4


class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.encoder = Encoder(1)
        self.avgpool = nn.AdaptiveMaxPool2d(output_size=1)

        self.classifier = nn.Sequential(
            nn.Linear(1024, 512),
            nn.Dropout(0.2),
            nn.Tanh(),
            nn.Linear(512, 64),
            nn.Dropout(0.2),
            nn.Tanh(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        enc_output = self.encoder(x)
        avg_pooled = self.avgpool(enc_output)
        avg_pooled = avg_pooled.view(avg_pooled.size(0), -1)
        output = self.classifier(avg_pooled)
        return output

In [ ]:
model = CustomModel().to('cuda')

In [ ]:
model

CustomModel(
  (encoder): Encoder(
    (enc1): Sequential(
      (0): Conv2d(1, 32, kernel_size=(9, 9), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(32, 32, kernel_size=(6, 6), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (enc2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(9, 9), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(6, 6), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=Fal

In [ ]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, roc_auc_score
from matplotlib import pyplot as plt
from torch import optim

model = CustomModel().to('cuda')

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 400

train_losses = []
val_losses = []
train_f1_scores = []
val_f1_scores = []
best_val_score = float('-inf')

for epoch in range(num_epochs):
    plt.clf()
    running_loss = 0.0
    model.train()
    train_predictions = []
    train_labels = []

    for i, data in enumerate(train_dataloader, 0):
        inputs1, _, labels = data
        inputs1, labels = inputs1.to('cuda'),  labels.to('cuda')


        optimizer.zero_grad()

        outputs = model(inputs1*255)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        train_predictions.extend(outputs.detach().cpu().numpy().flatten())
        train_labels.extend(labels.detach().cpu().numpy().flatten())

        running_loss += loss.item()

    train_loss = running_loss / (i + 1)
    train_losses.append(train_loss)
    train_f1 = probabilistic_f1(np.asarray(train_labels), np.round(train_predictions, decimals=2))
    train_f1_scores.append(train_f1)

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    val_predictions = []
    val_labels = []

    with torch.no_grad():
        for i, data in enumerate(val_dataloader, 0):
            inputs1, _, labels = data
            inputs1, labels = inputs1.to('cuda'),  labels.to('cuda')

            outputs = model(inputs1*255)
            loss = criterion(outputs, labels)

            val_predictions.extend(outputs.detach().cpu().numpy().flatten())
            val_labels.extend(labels.detach().cpu().numpy().flatten())

            val_loss += loss.item()

    val_loss = val_loss / (i + 1)
    val_losses.append(val_loss)
    val_f1 = probabilistic_f1(np.asarray(val_labels), np.asarray(val_predictions))
    val_f1_scores.append(val_f1)

    print(
        f'Epoch {epoch + 1}, Validation Loss: {round(val_loss, 3)}, F1 Score: {round(val_f1, 3)} Training Loss: {round(train_loss, 3)}, F1 Score: {round(train_f1, 3)}')
    if val_f1 > best_val_score:
      print("Saving model")
      best_val_score = val_f1
      torch.save(model.state_dict(), "gdrive/MyDrive/cnn_mg.pt")



Epoch 1, Validation Loss: 0.688, F1 Score: 0.517 Training Loss: 0.705, F1 Score: 0.513
Saving model
Epoch 2, Validation Loss: 0.669, F1 Score: 0.526 Training Loss: 0.692, F1 Score: 0.521
Saving model
Epoch 3, Validation Loss: 0.689, F1 Score: 0.486 Training Loss: 0.689, F1 Score: 0.524
Epoch 4, Validation Loss: 0.671, F1 Score: 0.534 Training Loss: 0.687, F1 Score: 0.529
Saving model
Epoch 5, Validation Loss: 0.685, F1 Score: 0.503 Training Loss: 0.679, F1 Score: 0.532
Epoch 6, Validation Loss: 0.68, F1 Score: 0.529 Training Loss: 0.669, F1 Score: 0.535
Epoch 7, Validation Loss: 0.695, F1 Score: 0.487 Training Loss: 0.664, F1 Score: 0.543
Epoch 8, Validation Loss: 0.703, F1 Score: 0.514 Training Loss: 0.659, F1 Score: 0.543
Epoch 9, Validation Loss: 0.737, F1 Score: 0.556 Training Loss: 0.656, F1 Score: 0.549
Saving model
Epoch 10, Validation Loss: 0.674, F1 Score: 0.542 Training Loss: 0.669, F1 Score: 0.555
Epoch 11, Validation Loss: 0.696, F1 Score: 0.526 Training Loss: 0.656, F1 Sco

KeyboardInterrupt: ignored

<Figure size 640x480 with 0 Axes>